In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

moviesFull = pd.read_csv('newdata/movies_metadata.csv',usecols=("genres","id" ,"title","tagline", "overview","production_companies"),
                          dtype={"tagline": "string", "id":"string", 'genres':"string", "title": "string", "tagline": "string","overview":"string", "production_companies" :"string"})
ratings = pd.read_csv('newdata/ratings.csv', usecols = ("userId", "movieId", "rating"), dtype={"userId": "string","movieId": "string","rating": "string"})
ratings = ratings.rename(columns={"movieId": "id"})

keywords = pd.read_csv('newdata/keywords.csv', usecols = ("id", "keywords"), dtype={"id": "string","keywords":"string"})
credits = pd.read_csv("newdata/credits.csv", usecols = ("cast", "id"), dtype={"cast": "string", "id": "string"})

complete =  pd.merge(moviesFull, ratings, on ="id")
complete =  pd.merge(complete,keywords, on ="id")
complete  = pd.merge(complete,credits, on ="id")


complete = complete.sort_values(by = 'userId')

complete  = complete.dropna()

complete  = complete.loc[:,['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview" ]]


In [2]:
import numpy as np
import ast


#used to filter the rows of a data
def condition(array):
    length = len(array[4])
    if(array[4][length-2:] == "[]"):
        return False
    length = len(array[5])
    if(array[5][length-2:] == "[]"):
        return False
    length = len(array[6])
    if(array[6][length-2:] == "[]"):
        return False
    length = len(array[7])
    if(array[7][length-2:] == "[]"):
        return False   
    length = len(array[8])
    if(array[8][length-4:]=="<NA>"):
        return False
    length = len(array[9])
    if(array[9][length-4:]=="<NA>"):
        return False 
    return True


#used to extract names from string of list of json formats
def populateNames(item):
    string  = item[1:-1]
    jsons = string.split("}, ")   
    names = ""
    cnt = 0
    for item in jsons:
        if(cnt == len(jsons)-1):
            tempDict = ast.literal_eval(item)
            names+=str(tempDict["name"])
        else:
            tempDict = ast.literal_eval(item+"}")
            names+=str(str(tempDict["name"])+" ")
        cnt += 1
    return names


def provideData(array):
    movieData = []
    movieData.append(int(array[0]))
    movieData.append(int(array[1]))
    movieData.append(float(array[2]))
    movieData.append(array[3])  

    movieData.append(populateNames(array[4]))
    movieData.append(populateNames(array[5]))
    movieData.append(populateNames(array[6]))
    movieData.append(populateNames(array[7]))

    movieData.append(str(array[8]))
    movieData.append(str(array[9]))
    return movieData
    


#convert the dataframe into an array
#and then build a dictionary
completeDict = dict()
completeArray = complete.to_numpy()
arrayOfUserIds = []


#get all unique user ids
lastId  = -1
for item in completeArray:
    if(item[0]!= lastId):
        arrayOfUserIds.append(item[0])
        lastId = item[0]


index  = 0
nofUsers = 5000
#5000 users are tested and potentially added to the dict
for i in range(0, nofUsers):
    completeDict[arrayOfUserIds[i]] = []
    for j in range(index, len(completeArray)):
        if completeArray[j][0] == arrayOfUserIds[i]:
            #this is where conditions are checked in completeArray[j]
            if(condition(completeArray[j])):
                #this is where data is tranformed
                transformed = provideData(completeArray[j])
                completeDict[arrayOfUserIds[i]].append(transformed)         
        else:
            #ignore if the number of ratings is too small
            if (len(completeDict[arrayOfUserIds[i]])<10):
                del completeDict[arrayOfUserIds[i]]
            index = j+1
            break


In [3]:
#save in a file so that cells below can run without running the above
import csv

with open("constructedData/constructedData.csv", "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview"])
    for item in completeDict.keys():
        writer.writerows(completeDict[item])


In [4]:
#this is a starting point if the data is already saved to the csv file
import csv
import pandas as pd

dataList =[]

with open("constructedData/constructedData.csv", 'r', encoding="utf-8") as read_obj:
    csv_reader = csv.reader(read_obj)
    dataList = list(csv_reader)

dataList = dataList[1:]


In [5]:
#movie id to list of ratings
movieDict = dict()

#user id to the rated movies by that user
userDict = dict()

#The list created by the constructed data is in order by user id
#this code makes a dictionary out of the data (user id to a list of text from all movies rated by the user)
#it also makes a dictionary of movies to their ratings
userId = -1
for row in dataList:
    if (row[0]!=userId):
        userId = row[0]
        userDict[row[0]] = [row]
    else:
        userDict[row[0]].append(row)

    if(row[1] in movieDict.keys()):
        movieDict[row[1]].append(row[2])
    else:
        movieDict[row[1]] = [row[2]]


In [6]:
from gensim.parsing.preprocessing import remove_stopwords

#dictionary of user id to a list of string of combined textual features for each movie 
#does not include ratings or movie id

combinedCorpus = dict()

i = 0
for key in userDict.keys():
    movieStrings = []
    for movieData in userDict[key]:
        movieString = ""
        #avoid the first three data points (user id, movieid, and rating)
        for index in range (3,len(movieData)):
            if(index!= len(movieData)-1):
                movieString+= movieData[index]+" "
            else:
                movieString+= movieData[index]
        cleaned = remove_stopwords(movieString)
        movieStrings.append(cleaned)
    combinedCorpus[key] = movieStrings

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import statistics
import math
import random
import copy
from scipy.stats import kurtosis
from scipy.stats import skew
import numpy as np

#analysis 
random.seed(1)

#get average rating for a single movie amoung all users who rated it
def getAverageMovieRatings(movieId):
    ret =0 
    cnt = 0
    for item in movieDict[movieId]:
        ret+= float(item)
        cnt+=1
    return (ret/cnt)


#get all the user ratings
def getUserRatings(userId):
    ret = []
    for item in userDict[userId]:
        ret.append(float(item[2]))
    return ret


X = []
y = []
userToPair = dict()
userToRandRating = dict()




#note: ratings and similairty scores for a given user need to be ordered the same:

for key in combinedCorpus.keys():

    count_matrix = CountVectorizer().fit_transform(combinedCorpus[key]).toarray().tolist()
    randIndex = random.randint(0, len(count_matrix)-1)
    randTestItem = count_matrix[randIndex]
    del count_matrix[randIndex]

    #find similarity with the count of each word between the random movie and the other movies
    cosine_sim = cosine_similarity(X = count_matrix ,Y = [randTestItem])

    ratings = copy.deepcopy(getUserRatings(key))
    randomRating = ratings[randIndex]
    userToRandRating[key] = randomRating
    del ratings[randIndex]


    similairities = np.reshape(cosine_sim,  (len(cosine_sim)))
    #can be saved for later:
    # averageRating =  sum(ratings)/(len(ratings))

    #populate features (more detail in paper)
    # features = []


    #need a data structure that keeps track of all the movies a user rated
    #dictionary of user to [sim, rating] pair


    #should also include average user rating for movies and avergae rating for the movie amoung all users
    for sim, rating in zip(similairities, ratings):
        X.append([sim, rating])
        y.append(randomRating)
        if key not in userToPair:
            userToPair[key] = [[sim, rating]]
        else:
            userToPair[key].append([sim, rating])




    #can be saved for later:
    # #average rating for the test movie for all users
    # #note: userDict[key][randIndex][1] is a movie id
    # features.append(getAverageMovieRatings(userDict[key][randIndex][1]))
    # #average rating for all movies rated by the given user given by "key"
    # features.append(averageRating)
    # #average cosine similairty score between the randomly chosen movie rated by the user given by "key"
    # #and the other movies rated by the same user
    # #what does this do???
    # features.append(sum(similairities)/(len(similairities)))


    # old features
    # X.append(features)
    # y.append(randomRating)



In [19]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

#(20,10,10,5)
#(10, 10, 10)
#(100, 100, 100)
layers = (10, 10, 10)
act = "tanh"
solve = "adam"

#note: used to be 1000 iterations
regr = MLPRegressor(hidden_layer_sizes=layers,activation =act, solver =solve,  max_iter=10000, random_state =1)

#fits the whole model 
regr.fit(X, y)


#note: this does not split training and test data
#there could also be a weigthed average applied

# there is no guaranteed order for the list of keys returned by the keys() function.
# In most cases, the key list is returned in the same order as the insertion,
# however, that behavior is NOT guaranteed and should not be depended on by your program.


userToAveRating = dict()

for key in userToPair.keys():
    predicted  = regr.predict(userToPair[key])
    sum = 0
    cnt = 0
    for item in predicted:
        sum+=item
        cnt+=1
    userToAveRating[key] = float(sum/cnt)



actualsList = []
predsList = []
for key in userToAveRating.keys():
    print("Pred: "+str(userToAveRating[key]) , "Actual: "+str(userToRandRating[key]))
    actualsList.append(userToRandRating[key])
    predsList.append(userToAveRating[key])


print("overall score:", r2_score(actualsList, predsList))
    
    

Pred: 3.728677439758823 Actual: 4.0
Pred: 3.669485185914497 Actual: 3.0
Pred: 3.5990810798103294 Actual: 4.0
Pred: 3.633247781464675 Actual: 4.0
Pred: 3.653925587058893 Actual: 4.0
Pred: 3.614452539593453 Actual: 3.0
Pred: 3.5339664226867336 Actual: 4.0
Pred: 3.576728094195063 Actual: 2.0
Pred: 3.405324038740336 Actual: 2.0
Pred: 3.5333394142423327 Actual: 3.0
Pred: 3.531173960191035 Actual: 2.5
Pred: 3.6176778570841197 Actual: 4.0
Pred: 3.608288735401633 Actual: 4.0
Pred: 3.5484705236166816 Actual: 3.0
Pred: 3.599015589077572 Actual: 3.5
Pred: 3.5368272802486533 Actual: 4.0
Pred: 3.398817791687401 Actual: 3.5
Pred: 3.5981154258499544 Actual: 1.0
Pred: 3.5358150574438127 Actual: 3.0
Pred: 3.6502175500622664 Actual: 3.5
Pred: 3.277593485839498 Actual: 3.0
Pred: 3.5667815404754326 Actual: 3.0
Pred: 3.5693537506650714 Actual: 4.0
Pred: 3.5088356003521075 Actual: 1.0
Pred: 3.532022615800575 Actual: 3.0
Pred: 3.735231463727407 Actual: 4.0
Pred: 3.4138011036218607 Actual: 3.0
Pred: 3.5306531